# 抖音用户数据分析
$——3.数据挖掘探索—3.1 聚类分析$


在对用户、作者、作品进行简单的描述性统计分析与可视化展示后，我们尝试通过一些数据挖掘方法对数据进一步探究   
对于抖音平台本身而言，如何对用户进行分类，或者分级，然后差异化的提供服务，是一个非常重要的方向  
对于商务合作和广告投放者而言，如何对作者进行分类，如何选择合作的作者，也是有一定价值的问题  
3.1中将针对用户和作者的数据特征，使用kmeans聚类算法量化的将两个群体进行分类

In [1]:
import numpy as np
import pandas as pd

from pyecharts.charts import *
from pyecharts import options as opts

from sklearn.cluster import KMeans
import joblib
from sklearn import metrics
from scipy.spatial.distance import cdist

## 1. 数据读取与数据处理

### 1.1 数据读取

In [2]:
user_feature = pd.read_csv('用户特征.csv', index_col=0)
author_feature = pd.read_csv('作者特征.csv', index_col=0)

### 1.2 数据处理

用户聚类可以服务于平台对用户分级，探索性的分析用户特点，但在二、数据可视化分析中，可以看到有一部分用户使用程度平台低 ，浏览少，不点赞，对这样的用户进行聚类分析是无效多余的，增加筛选认为至少观看过一个完整短视频且有一定浏览量的用户才具有分析意义

In [3]:
user_data = user_feature[(user_feature['完整观看数']>=1)&(user_feature['浏览量']>=5)]
print(len(user_data)/len(user_feature))

0.7097514856834144


而在对作者的考量上，聚类的结果是服务于商务合作和广告投放，此时核心是浏览量  
而大部分的作者总浏览量非常小，这些作者是无需考虑的，故进行筛选

In [4]:
author_data = author_feature[(author_feature['总观完量']>=1)&(author_feature['总浏览量']>=3)]
print(len(author_data)/len(author_feature))

0.2990244347629775


## 2. 聚类方法与定义

### 2.1 聚类方法

关键参数：
- init='k-means++'  (其实默认就是这个，可以不用写)
        对于kmeans算法，初始中心的选取是至关重要的。  
        kmeans的随机选取方法可能会出现初始中心过于接近，导致迭代结果收敛慢，效果差  
        kmeans++通过逐个选取中心，并优先选取距离较远的中心来优化初始中心的选择
- n_clusters: 聚类数  
        聚类数的确定通过综合不同指标，应用肘部法则进行判断

评价指标
- SSE: 误差平方和  
        当前迭代得到的中心位置到各自中心点簇的欧式距离总和
- 轮廓系数：sc轮廓系数 
        都是结合了聚类的类内凝聚度和类间分离度
        sc轮廓系数∈[-1, 1]，越接近1越好

### 2.2 相关函数定义

**Kmeans**

In [10]:
def km(data, name):
    K = range(2, 10) # K值选取范围
    X = data # 数据
    # scores = { 'SSE': [], 'sc': [], 'sse': []}
    scores = {'sc': [], 'sse': []}
    for _k in K:
        # 初始化模型并进行聚类 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        kmeans = KMeans(n_clusters=_k, init='k-means++', random_state=0)
        kmeans.fit(X)
        _y = kmeans.predict(X) # 预测结果
        # 计算模型评估指标 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        sse = sum(np.min(cdist(X,kmeans.cluster_centers_,'euclidean'),axis=1))/X.shape[0]
        sc = metrics.silhouette_score(X, _y) # 计算轮廓系数
        joblib.dump(kmeans, f'{name}{_k}聚类.model')
        # 储存评估值 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        # scores['SSE'].append(SSE)
        scores['sse'].append(sse)
        scores['sc'].append(sc)
        print(f'聚{_k}类计算完成', end='\t')
    joblib.dump(scores, f'{name}聚类指标.score')
    print('指标储存完毕')
    return scores

**绘制sse和sc曲线**

In [11]:
def draw(k, sse, sc):
    chart = (
        Line(init_opts=opts.InitOpts(
            theme='light',
            width='350px',
            height='350px'
        ))
        .add_xaxis(k)
        .add_yaxis('sse', sse, yaxis_index=0, label_opts=opts.LabelOpts(is_show=False))
        .add_yaxis('sc', sc, yaxis_index=1, label_opts=opts.LabelOpts(is_show=False))
        .extend_axis(yaxis=opts.AxisOpts())
        .set_global_opts(
            title_opts=opts.TitleOpts(title='聚类效果'),
            xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=True),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                axistick_opts=opts.AxisTickOpts(is_show=True),
                splitline_opts=opts.SplitLineOpts(is_show=True),
            ),
        )
    )
    return chart

## 3. 用户特征聚类

### 3.1 模型训练与保存

In [12]:
user_score = km(user_data, '用户')

聚2类计算完成	聚3类计算完成	聚4类计算完成	聚5类计算完成	聚6类计算完成	聚7类计算完成	聚8类计算完成	聚9类计算完成	指标储存完毕


### 3.2 聚类k值选择

In [14]:
user_score =  joblib.load(f'用户聚类指标.score')
draw([str(x) for x in range(2,10)], user_score['sse'], user_score['sc']).render_notebook()

通过综合肘部法则和sc值，选择$k=4$作为用户聚类模型

### 3.3 聚类结果

In [15]:
user_km = joblib.load(f'用户4聚类.model')
user_centers = pd.DataFrame(user_km.cluster_centers_, columns=user_feature.columns)
user_centers['人数']=pd.Series(user_km.predict(user_data)).value_counts()
user_centers

,浏览量,点赞量,观看作者数,观看作品数,观看作品平均时长,观看配乐数,完整观看数,去过的城市数,观看作品城市数,人数
0,65.559893,0.651947,63.616853,65.559040,11.175167,58.035413,27.440747,1.274133,47.094827,9366
1,381.603365,3.012019,353.144231,381.598558,10.969891,276.408654,92.346154,1.324519,140.896635,416
2,162.612409,1.259288,154.847185,162.610494,11.078860,131.901187,54.075450,1.293757,88.666794,2610
3,16.791653,0.190634,16.512585,16.791518,11.315000,15.934307,8.124806,1.170288,14.572070,29648


## 4. 作者特征聚类

### 4.1 模型训练与保存

In [16]:
author_score = km(author_data, '作者')

聚2类计算完成	聚3类计算完成	聚4类计算完成	聚5类计算完成	聚6类计算完成	聚7类计算完成	聚8类计算完成	聚9类计算完成	指标储存完毕


### 4.2 聚类k值选择

In [17]:
author_score =  joblib.load(f'作者聚类指标.score')
draw([str(x) for x in range(2,10)], author_score['sse'], author_score['sc']).render_notebook()

通过综合肘部法则和sc值，选择$k=4$作为用作者聚类模型

### 4.3 聚类效果

In [18]:
author_km = joblib.load(f'作者4聚类.model')
author_centers = pd.DataFrame(author_km.cluster_centers_, columns=author_feature.columns)
author_centers['人数'] = pd.Series(author_km.predict(author_data)).value_counts()
author_centers

,总浏览量,总点赞量,总观完量,总作品数,作品平均时长,使用配乐数量,发布作品日数,创作活跃度(日),去过的城市数,人数
0,11.973167,0.115981,5.019598,3.636747,10.783111,3.198138,3.636625,11.301008,1.114046,57356
1,376.060858,3.428769,165.146611,20.035961,11.126590,13.811895,20.034578,24.224066,1.289073,723
2,1092.821053,8.347368,461.315789,31.126316,11.165662,19.957895,31.126316,27.926316,1.326316,95
3,117.075717,1.148983,51.548395,12.432737,10.993801,9.434942,12.431267,21.253124,1.236217,4079


## 总结
聚类的结果解释性较为明显，其核心与浏览量相关，提供了一定数据特征下的量化分类作用  